In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [3]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [4]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {}
for i in range(len(world_set)):
    vocab[world_set[i]] = i

In [13]:
print(vocab)

{'o': 0, 'm': 1, '.': 2, 'k': 3, 'f': 4, 'r': 5, 'n': 6, 'e': 7, ' ': 8, "'": 9, 'g': 10, 'w': 11, 'u': 12, 'c': 13, 'h': 14, 't': 15, 'B': 16, 'd': 17, 'p': 18, 'a': 19, ',': 20, 'i': 21, 'l': 22, 's': 23, 'y': 24, 'b': 25}


In [5]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [6]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [ ]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

In [ ]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

In [8]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[data] for data in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTensor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\tpgns\AppData\Local\Temp\ipykernel_23500\3292177215.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  X = torch.FloatTensor(x_one_hot)


In [9]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [ ]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

In [27]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 5, 21, 13,  3,  8, 11, 19, 22, 22, 23])


In [10]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim)
  
  def forward(self, x):
      x, hidden = self.rnn(x)
      N, T, H = x.shape
      x = x.contiguous().view(N*T, H)
      x = self.fc(x)
      x = x.view(N, T, H)
      return x, hidden

In [11]:
net = Net(vocab_size, hidden_size, 2)

In [12]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [13]:
##14. 출력 크기 점검
outputs, h = net(X)
print(outputs.shape)
print(h.shape)

torch.Size([188, 10, 26])
torch.Size([2, 188, 26])


In [ ]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs, _ = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.reshape(-1,26), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

In [15]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to khow us how badly we want thinksm"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [16]:
sentence = ("The only way to do great work is to love what you do. "
            "In the middle of difficulty lies opportunity, so Believe you can and you're halfway there. "
            "Success is not final, failure is not fatal. It is the courage to continue that counts")

In [20]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {word:id for id, word in enumerate(world_set)}
vocab_size = len(vocab)

In [36]:
hidden_size = 10
sequence_length = 20  
learning_rate = 0.01

In [ ]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+1+sequence_length]
  print(i, x_str, "->", y_str)

  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

In [38]:
x_one_hot = [np.eye(vocab_size)[data] for data in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [39]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers, vocab_size):
    super(Net, self).__init__()
    self.vocab_size = vocab_size
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size)
  
  def forward(self, x):
      x, hidden = self.rnn(x)
      N, T, H = x.shape
      x = x.contiguous().view(N*T, H)
      x = self.fc(x)
      x = x.view(N, T, self.vocab_size)
      return x, hidden

In [40]:
net = Net(vocab_size, hidden_size, 2, vocab_size)

In [41]:
criterion = torch.nn.CrossEntropyLoss()


optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
for i in range(500):
    optimizer.zero_grad()
    outputs, _ = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.reshape(-1,vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

In [43]:
predict_str

'he fnly tay th do great work is to love what you do. In the middle of difficulty lies opportunity, so Believe you can and you re halfway there. Success is not final, failure is not fital. It is the courege to continue that county'